In [ ]:
import setproctitle

setproctitle.setproctitle("python3.12")
import os
import sys

sys.path.append("..")

from transformers import set_seed
from sentence_transformers import SentenceTransformer
from sentence_transformers.SentenceTransformer import SentenceTransformer
from benchmark_generator.context.utils.jsonl import read_jsonl, write_jsonl
from tqdm import tqdm

os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
os.environ["CUDA_VISIBLE_DEVICES"] = "5"
set_seed(42, deterministic=True)

embedding_model = SentenceTransformer("../models/stella", local_files_only=True)

In [2]:
EMBEDDING_MAX_TOKENS = 512

# Splitter for Schema Summaries

In [22]:
type = "standard"
name = f"adventure_{type}"
contents = read_jsonl(f"summaries/{type}/{name}.jsonl")

In [ ]:
processed_contents = []
unique_tables = sorted(set([row["table"] for row in contents]))
tokenizer = embedding_model.tokenizer
for table in tqdm(unique_tables):
    schema_summary = [summary for summary in contents if summary["table"] == table][0]["summary"]
    column_summaries = schema_summary.split(" | ") 

    col_idx = 0
    while col_idx < len(column_summaries):
        processed_summary = column_summaries[col_idx]

        while (col_idx + 1) < len(column_summaries):
            temp = processed_summary + " | " + column_summaries[col_idx + 1]
            if len(tokenizer.encode(temp)) < EMBEDDING_MAX_TOKENS:
                processed_summary = temp
                col_idx += 1
            else:
                break

        col_idx += 1
        processed_contents.append({
            "table": table,
            "summary": processed_summary,
        })

In [ ]:
print(f"Num of content summaries (BEFORE): {len(contents)}")
print(f"Num of content summaries (AFTER): {len(processed_contents)}")

In [25]:
write_jsonl(processed_contents, f"summaries/{type}/{name}_splitted.jsonl")